Este colab contém duas implementações do modelo para resolver o Problema do Caixeiro Viajante

Para um problema com $n$ cidades, temos $i,j \in \{1,2 \dots ,n\}$.

$min \sum_{i,j} d_{ij}*X_{ij}$

$\sum_i X_{ij} = 1 \quad \forall \quad j$

$\sum_j X_{ij} = 1 \quad \forall \quad i$

$X_{jj} = 0 \quad \forall \quad i$

$X_{ij} \in \{0,1\} \quad \forall \quad i,j$

> O método de Dantzig, com a eliminação de sub-rotas

$\sum_{i,j} X_{ij} \leq |S| - 1 \quad \forall \quad i,j \in S$

> O Método MTZ

$u_i - u_j \leq n*(1-X_{ij}) - 1 \quad \forall \quad j, i \geq 2, i \neq j $

$u_{i} \geq 0 \quad \forall \quad i$


As instâncias foram retiradas do banco de dados TSPLIB

http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/



In [1]:
%pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB
import time

Looking in indexes: https://pypi.gurobi.com
     |████████████████████████████████| 11.1 MB 19.3 MB/s 


In [2]:
from google.colab import files
uploaded = files.upload()

Saving ftv33.atsp to ftv33.atsp


In [3]:
def find_next(X, j):
	for i in range(n):
		if round(X[j,i].X) == 1:
			next = i		
	return next

def ver_subtour(X,j):
	subtour = [j]
	next = find_next(X,j)
	while next != j:
		subtour.append(next)
		next = find_next(X,next)
	return subtour

def escreve_sol(X):
	solucao = []
	for j in range(n):
		j_sol = 0
		for i in range(len(solucao)):
			if j in solucao[i]:
				j_sol = 1			
		if j_sol == 0:
			subtour = ver_subtour(X,j)
			solucao.append(subtour)
	return solucao

In [4]:
f = open('ftv33.atsp', 'r')
for i in range(3):
  line = f.readline()
line = f.readline()
n = int(line.split()[1])
for i in range(3):
  line = f.readline()
d = []
i = 0
d.append([])
while i < n:
  line = f.readline()
  for num in line.split():
    if len(d[i]) != n:
      d[i].append(int(num))
    if len(d[i]) == n:
      d.append([])
      i += 1

In [5]:
model = gp.Model("Dantzig")
gp.setParam('LogToConsole', 0)

X = {}
X = model.addVars(n,n,vtype=GRB.BINARY)

distance = sum(X[j,i]*d[j][i] for j in range(n) for i in range(n))
model.setObjective(distance, GRB.MINIMIZE)

model.addConstrs(sum(X[j,i] for i in range(n)) == 1 for j in range(n))
model.addConstrs(sum(X[j,i] for j in range(n)) == 1 for i in range(n))
model.addConstrs(X[j,j] == 0 for j in range(n))
model.update()

start_time = time.time()

while True:
  model.optimize()	
  solucao = escreve_sol(X)
  n_rotas = len(solucao)
  print("\nNúmero de rotas: "+str(n_rotas))
  print(solucao)
  print("Valor: "+str(model.objVAL))
  if n_rotas != 1:
    for r in range(n_rotas):
      model.addConstr(sum(X[j,i] for j in solucao[r] for i in solucao[r]) <= len(solucao[r]) - 1)
    model.update()
  else:
    break

end_time = time.time()
print("\nTempo total: "+str(round(end_time-start_time,2)))

Restricted license - for non-production use only - expires 2022-01-13

Número de rotas: 8
[[0, 1, 2, 3], [4, 5, 6], [7, 8, 10, 9, 32], [11, 31, 18, 17], [12, 13], [14, 16, 15], [19, 21, 20, 22, 26, 27, 28, 29, 25, 24, 23], [30, 33]]
Valor: 1185.0

Número de rotas: 8
[[0, 13], [1, 33, 30, 2, 3], [4, 32, 7], [5, 6], [8, 10, 9, 12, 14, 15, 16, 25, 24, 23, 19, 31, 18, 17, 11], [20, 21], [22, 26], [27, 28, 29]]
Valor: 1252.0

Número de rotas: 3
[[0, 13, 12, 14, 15, 16, 1, 25, 24, 23, 19, 31, 18, 17, 11, 8, 10, 9, 32, 7, 4, 6, 5, 30, 33, 2, 3], [20, 21, 22], [26, 27, 28, 29]]
Valor: 1263.0

Número de rotas: 2
[[0, 13, 12, 14, 15, 16, 1, 25, 24, 23, 20, 21, 31, 18, 19, 17, 11, 8, 10, 9, 32, 7, 4, 6, 5, 30, 33, 2, 3], [22, 27, 28, 29, 26]]
Valor: 1284.0

Número de rotas: 1
[[0, 13, 12, 14, 15, 16, 1, 25, 24, 23, 26, 27, 28, 29, 22, 20, 21, 31, 18, 19, 17, 11, 8, 10, 9, 32, 7, 4, 6, 5, 30, 33, 2, 3]]
Valor: 1286.0

Tempo total: 0.16


In [6]:
model = gp.Model("MTZ")
gp.setParam('LogToConsole', 0)

X = {}
X = model.addVars(n,n,vtype=GRB.BINARY)
u = {}
u = model.addVars(n,vtype=GRB.CONTINUOUS,lb=0)

distance = sum(X[j,i]*d[j][i] for j in range(n) for i in range(n))
model.setObjective(distance, GRB.MINIMIZE)

model.addConstrs(sum(X[j,i] for i in range(n)) == 1 for j in range(n))
model.addConstrs(sum(X[j,i] for j in range(n)) == 1 for i in range(n))
model.addConstrs(X[j,j] == 0 for j in range(n))

model.addConstrs(u[i] - u[j] <= n*(1-X[i,j]) - 1 
                 for j in range(n) for i in range(1,n) if i != j)

model.update()

start_time = time.time()
model.optimize()	
solucao = escreve_sol(X)
n_rotas = len(solucao)
print("\nNúmero de rotas: "+str(n_rotas))
print(solucao)
print("Valor: "+str(model.objVAL))
end_time = time.time()
print("\nTempo total: "+str(round(end_time-start_time,2)))


Número de rotas: 1
[[0, 13, 12, 14, 15, 16, 1, 25, 24, 23, 27, 28, 29, 26, 22, 20, 21, 31, 18, 19, 17, 11, 8, 10, 9, 32, 7, 4, 6, 5, 30, 33, 2, 3]]
Valor: 1286.0

Tempo total: 0.73
